Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size


In [109]:
import pandas as pd 
import dask.dataframe as dd
import modin.pandas as mpd
import ray.data as rd
import ray
import os
import time

#Calculating the size of the file
os.path.getsize('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv')

#Reading Using Pandas 
start = time.time()
pd_df = pd.read_csv('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv')
end = time.time()
print("reading time for a CSV file in pandas: ",(end-start),"sec")


#Reading using Dask
start = time.time()
dd_df = dd.read_csv('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv')
end = time.time()
print("reading time for a CSV file in Dask: ",(end-start),"sec")

#Reading Using Modin
ray.init()
start = time.time()
mpd_df = mpd.read_csv('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv')
emd = time.time()
print("reading time for a CSV file in Modin: ",(end-start),"sec")
mpd.shutdown_ray()

#Reading Using Ray
start = time.time()
df = rd.read_csv('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv')
end = time.time()
print("reading time for a CSV file in Ray: ",(end-start),"sec")


reading time for a CSV file in pandas:  0.09613394737243652 sec
reading time for a CSV file in Dask:  0.9768166542053223 sec


2023-08-13 01:35:27,894	INFO read_api.py:374 -- To satisfy the requested parallelism of 200, each read task output will be split into 50 smaller blocks.


reading time for a CSV file in Ray:  2.0164434909820557 sec


In [47]:
#Delete special characters names and white spaces in columns
dd_df.columns = dd_df.columns.str.replace('?,@,#,$,%,&,*', ' ')
dd_df.columns = dd_df.columns.str.replace(' ', '')

#Print the updated column names
print(dd_df.columns)

Index(['SummonsNumber', 'PlateID', 'RegistrationState', 'PlateType',
       'IssueDate', 'ViolationCode', 'VehicleBodyType', 'VehicleMake',
       'IssuingAgency', 'StreetCode1', 'StreetCode2', 'StreetCode3',
       'VehicleExpirationDate', 'ViolationLocation', 'ViolationPrecinct',
       'IssuerPrecinct', 'IssuerCode', 'IssuerCommand', 'IssuerSquad',
       'ViolationTime', 'TimeFirstObserved', 'ViolationCounty',
       'ViolationInFrontOfOrOpposite', 'HouseNumber', 'StreetName',
       'IntersectingStreet', 'DateFirstObserved', 'LawSection', 'SubDivision',
       'ViolationLegalCode', 'DaysParkingInEffect', 'FromHoursInEffect',
       'ToHoursInEffect', 'VehicleColor', 'UnregisteredVehicle?',
       'VehicleYear', 'MeterNumber', 'FeetFromCurb', 'ViolationPostCode',
       'ViolationDescription', 'NoStandingorStoppingViolation',
       'HydrantViolation', 'DoubleParkingViolation', 'Latitude', 'Longitude',
       'CommunityBoard', 'CommunityCouncil', 'CensusTract', 'BIN', 'BBL',
      

In [120]:
%%writefile park_file.yaml
file_type: csv
dataset_name: Parking_Violations_Year_2016.csv
file_name: Parking_Violations_Year_2016.csv
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - SummonsNumber
    - PlateID
    - RegistrationState
    - PlateType
    - IssueDate
    - ViolationCode
    - VehicleBodyType
    - VehicleMake
    - IssuingAgency
    - StreetCode1
    - StreetCode2
    - StreetCode3
    - VehicleExpirationDate
    - ViolationLocation
    - ViolationPrecinct
    - IssuerPrecinct
    - IssuerCode
    - IssuerCommand
    - IssuerSquad
    - ViolationTime
    - TimeFirstObserved
    - ViolationCounty
    - ViolationInFrontOfOrOpposite
    - HouseNumber
    - StreetName
    - IntersectingStreet
    - DateFirstObserved
    - LawSection
    - SubDivision
    - ViolationLegalCode
    - DaysParkingInEffect
    - FromHoursInEffect
    - ToHoursInEffect
    - VehicleColor
    - UnregisteredVehicle
    - VehicleYear
    - MeterNumber
    - FeetFromCurb
    - ViolationPostCode
    - ViolationDescription
    - NoStandingorStoppingViolation
    - HydrantViolation
    - DoubleParkingViolation
    - Latitude
    - Longitude
    - CommunityBoard
    - CommunityCouncil
    - CensusTract
    - BIN
    - BBL
    - NTA
    

Writing park_file.yaml


In [130]:
#Read config file
import testutility as util
config_data = util.read_config_file("park_file.yaml")

config_data['inbound_delimiter']

','

In [ ]:
#inspecting data of config file
config_data

In [56]:
#Reading using Pandas
sample = pd.read_csv('C:/Users/yomna/week6/Parking_Violations_Year_2016.csv',delimiter=',', nrows=5)
sample.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1363745270,GGY6450,99,PAS,07/09/2015,46,SDN,HONDA,P,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1363745293,KXD355,SC,PAS,07/09/2015,21,SUBN,CHEVR,P,55730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1363745438,JCK7576,PA,PAS,07/09/2015,21,SDN,ME/BE,P,42730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1363745475,GYK7658,NY,OMS,07/09/2015,21,SUBN,NISSA,P,58130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1363745487,GMT8141,NY,PAS,07/09/2015,21,P-U,LINCO,P,58130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "C:/Users/yomna/week6/" + config_data['file_name']
delimiter = config_data['inbound_delimiter']

df = pd.read_csv(source_file, sep=delimiter, nrows=5)

df.columns = df.columns.str.replace('?', ' ')
df.columns = df.columns.str.replace(' ', '')
#df.head()

print(df.columns)

Index(['SummonsNumber', 'PlateID', 'RegistrationState', 'PlateType',
       'IssueDate', 'ViolationCode', 'VehicleBodyType', 'VehicleMake',
       'IssuingAgency', 'StreetCode1', 'StreetCode2', 'StreetCode3',
       'VehicleExpirationDate', 'ViolationLocation', 'ViolationPrecinct',
       'IssuerPrecinct', 'IssuerCode', 'IssuerCommand', 'IssuerSquad',
       'ViolationTime', 'TimeFirstObserved', 'ViolationCounty',
       'ViolationInFrontOfOrOpposite', 'HouseNumber', 'StreetName',
       'IntersectingStreet', 'DateFirstObserved', 'LawSection', 'SubDivision',
       'ViolationLegalCode', 'DaysParkingInEffect', 'FromHoursInEffect',
       'ToHoursInEffect', 'VehicleColor', 'UnregisteredVehicle', 'VehicleYear',
       'MeterNumber', 'FeetFromCurb', 'ViolationPostCode',
       'ViolationDescription', 'NoStandingorStoppingViolation',
       'HydrantViolation', 'DoubleParkingViolation', 'Latitude', 'Longitude',
       'CommunityBoard', 'CommunityCouncil', 'CensusTract', 'BIN', 'BBL',
       

In [132]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [126]:
print("columns of files are:" ,dd_df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct',
       'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad',
       'Violation Time', 'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation', 'Latitud

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

In [133]:
import datetime
import csv
import gzip

#Creating a pipe-separated in gz format
df.to_csv('Parking_Violations_Year_2016.csvt.gz', sep='|', compression='gzip', index=False)

In [ ]:
import os

#Creating a summary file
file_size = os.path.getsize('your_file.csv')
num_rows = len(df_pandas)
num_cols = len(df_pandas.columns)

print("Total number of rows:", num_rows)
print("Total number of columns:", num_cols)
print("File size:", file_size)